**Facultad de Ciencias Exactas, Ingeniería y Agrimensura - UNR**

Tecnicatura Universitaria en Inteligencia Artificial

# Procesamiento del Lenguaje Natural - Trabajo Práctico N°: 1.

Integrantes:
- Pace, Bruno. Legajo: P-5295/7.
- Sancho Almenar, Mariano. Legajo: S-5778/9.

In [ ]:
# datasets
import pandas as pd

# modelado
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk
import spacy
from sentiment_analysis_spanish import sentiment_analysis

## Modelado
- Se utiliza transformers para clasificar el estado de ánimo de la persona. Este puede ser: "Alegre", "Melancólico" o "Ni fu ni fa".

In [ ]:
# Cargamos el tokenizador y el modelo.
model_name: str = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer: BertTokenizer = BertTokenizer.from_pretrained(model_name)
model: BertForSequenceClassification = BertForSequenceClassification.from_pretrained(model_name)


# Creación de pipeline.
nlp: pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
def get_sentiment(text: str) -> str:
    """
    Retorna el sentimiento de un texto, mediante el uso de transformers.
    """
    label: str = nlp(text)[0]['label']
    score: float = nlp(text)[0]['score']

    if label in ['5 stars', '4 stars']:
        print(score)
        return 'Alegre'
    elif label == '3 stars':
        print(score)
        return 'Ni fu ni fa'
    elif label in ['2 stars', '1 star']:
        print(score)
        return 'Melancólico'

In [ ]:
test_list: list[str] = [
    # Frases alegres
    "No puedo esperar para sentir la brisa del mar y disfrutar del sol en la playa.",
    "Tengo muchas ganas de jugar al voleibol en la arena con mis amigos.",
    "Estoy emocionado por las cenas al aire libre y las caminatas por la playa.",

    # Frases neutras
    "Voy a la playa, pero espero que el clima no arruine mis planes.",
    "Me gustaría leer un buen libro si no puedo salir por la lluvia.",
    "Aún estoy organizando lo que haré durante mis vacaciones.",

    # Frases melancólicas
    "No estoy seguro de que la lluvia me deje disfrutar de mis vacaciones.",
    "Me siento un poco decepcionado por la posibilidad de mal tiempo.",
    "Ojalá pudiera cambiar el clima para disfrutar más de mis días libres.",

    # Frases de frustración
    "No sé qué hacer si llueve todos los días de mis vacaciones.",
    "Quiero disfrutar, pero la lluvia arruina todo.",
    "Me molesta pensar que no podré hacer nada divertido en la playa.",

    # Frases optimistas
    "Si llueve, siempre hay buenas películas que puedo ver.",
    "Aprovecharé los días de lluvia para leer más libros que tengo pendientes.",
    "Estoy listo para adaptarme; siempre puedo jugar a algo divertido en casa."
]

for text in test_list:
  print(f'{text} -> {get_sentiment(text)}')

A fin de realizar una comparación, utilizamos sentiment-analysis sobre la misma lista de test.

In [ ]:
sentiment = sentiment_analysis.SentimentAnalysisSpanish()

for texto in test_list:
  print(f'{texto} -> {sentiment.sentiment(texto)}')

El rendimiento obtenido es inferior, por eso se optó por los transformers.